In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math

In [2]:
train = pd.read_csv('train.csv')
train = train.loc[train['text'].drop_duplicates().index]

In [3]:
tkeyword = train[train['target']==1][['id','keyword']].groupby('keyword').agg('count')
tkeyword.columns = ['keyword_t_count']
tkeyword.reset_index(inplace=True)

In [4]:
fkeyword = train[train['target']==0][['id','keyword']].groupby('keyword').agg('count')
fkeyword.columns = ['keyword_f_count']
fkeyword.reset_index(inplace=True)

In [5]:
keyword = tkeyword.merge(fkeyword)

In [6]:
keyword['keyword_metric']= keyword['keyword_t_count'] - keyword['keyword_f_count']
keyword['keyword_total']= keyword['keyword_t_count'] + keyword['keyword_f_count']
keyword['keyword_metric'] = keyword['keyword_metric']/keyword['keyword_total']
keyword = keyword.sort_values(by='keyword_total')
keyword['keyword_total'] = keyword['keyword_total'].apply(lambda x: 1 if x<20 else math.log(x+10,20))
scaler = MinMaxScaler()
x = keyword['keyword_total'].values.reshape(-1, 1)
keyword['keyword_total'] = scaler.fit_transform(x)
keyword['keyword_metric'] = keyword['keyword_metric'] * keyword['keyword_total']

In [7]:
keyword.drop('keyword_total',axis=1,inplace=True)

In [8]:
keyword['keyword_metric'].max(),keyword['keyword_metric'].min()

(0.8403855539409509, -0.8802189264555614)

In [9]:
keyword.nlargest(10,columns='keyword_metric')

,keyword,keyword_t_count,keyword_f_count,keyword_metric
150,outbreak,38,1,0.840386
149,oil%20spill,37,1,0.819880
202,typhoon,35,1,0.777615
184,suicide%20bombing,31,1,0.687590
183,suicide%20bomber,30,1,0.663827
144,nuclear%20disaster,31,3,0.641872
93,evacuated,32,4,0.640389
159,razed,31,4,0.618401
163,rescuers,29,3,0.595911
211,wildfire,29,4,0.573250


In [35]:
keyword.nsmallest(10,columns='keyword_metric')

,keyword,keyword_t_count,keyword_f_count,keyword_metric
28,body%20bags,1,40,-0.880219
167,ruin,1,36,-0.798962
216,wrecked,3,36,-0.749533
116,harm,4,37,-0.744801
18,blazing,1,33,-0.733567
170,screaming,2,34,-0.731873
95,explode,3,35,-0.728782
7,armageddon,5,37,-0.719660
26,body%20bag,1,32,-0.710830
198,traumatised,2,33,-0.710016


In [10]:
keyword.to_csv('data_process/keyword.csv',index=False)